In [1]:
import numpy as np
from daskms import xds_from_table,xds_from_ms
# import svdcompressor
# import svdaverager
from casacore.tables import table
import warnings
# import msodify
import dask.array as da
import compress
import archive
import zarr
import decompress


In [2]:
msfile = '/home/mukundi/averaging/stimela_scripts/msdir/sim-svd-test.ms'
mssub = '/home/mukundi/averaging/stimela_scripts/msdir/sim-simms.ms::ANTENNA'

In [3]:
antlist = [[0,1],
           [1,2],
           ]
# compress.compress_visdata(msfile,correlation='XX,XY,YX,YY',compressionrank=1,outcol='COMPRESSED_DATA',antlist=antlist,zarr_output_path=None)
archive.archive_visdata(msfile,correlation='XX,XY,YX,YY',compressionrank=10,antlist=antlist)

11:19 16:07:27-BDSVD-INFO    | Decomposing visibilities for DATA_DESC_ID 0. FIELD_ID 0, and SCAN_NUMBER 1.
11:19 16:07:27-BDSVD-INFO    | The following correlations are available:['XX', 'XY', 'YX', 'YY']
11:19 16:07:27-BDSVD-INFO    | The user has selected the following correlations:['XX', 'XY', 'YX', 'YY']
11:19 16:07:28-BDSVD-INFO    | Decomposing visibility data for baseline M000-M001 and correlation XX
11:19 16:07:28-BDSVD-INFO    | Decomposing visibility data for baseline M000-M001 and correlation XY
11:19 16:07:28-BDSVD-INFO    | Decomposing visibility data for baseline M000-M001 and correlation YX
11:19 16:07:28-BDSVD-INFO    | Decomposing visibility data for baseline M000-M001 and correlation YY
11:19 16:07:28-BDSVD-INFO    | Decomposing visibility data for baseline M001-M002 and correlation XX
11:19 16:07:28-BDSVD-INFO    | Decomposing visibility data for baseline M001-M002 and correlation XY
11:19 16:07:28-BDSVD-INFO    | Decomposing visibility data for baseline M001-M002 and

In [3]:
zarr_file = "/home/mukundi/kundikat/visco/visco/Output/Compressed_Data.zarr"

In [5]:
dec = decompress.decompress_visdata(zarr_file)

11:19 16:01:20-BDSVD-INFO    | Decompressed visibility data for baseline M000-M001, correlation XX
11:19 16:01:21-BDSVD-INFO    | Decompressed visibility data for baseline M000-M001, correlation XY
11:19 16:01:21-BDSVD-INFO    | Decompressed visibility data for baseline M000-M001, correlation YX
11:19 16:01:21-BDSVD-INFO    | Decompressed visibility data for baseline M000-M001, correlation YY
11:19 16:01:21-BDSVD-INFO    | Decompressed visibility data for baseline M001-M002, correlation XX
11:19 16:01:21-BDSVD-INFO    | Decompressed visibility data for baseline M001-M002, correlation XY
11:19 16:01:21-BDSVD-INFO    | Decompressed visibility data for baseline M001-M002, correlation YX
11:19 16:01:21-BDSVD-INFO    | Decompressed visibility data for baseline M001-M002, correlation YY
11:19 16:01:21-BDSVD-INFO    | Successfully decompressed visibility data from /home/mukundi/kundikat/visco/visco/Output/Compressed_Data.zarr


In [12]:
tab = xds_from_ms('/home/mukundi/kundikat/visco/visco/decompressed.ms')[0]
tab.DECOMPRESSED_DATA.shape

(6048, 24, 4)

In [18]:
zarr_open = zarr.open(zarr_file)
decomp = zarr_open["DECOMPOSITIONS"]
decomp['M000-M001']['XX']['U'][:]
ant = zarr_open["ANTENNA2"]
ant[:].nbytes/(1024)
decomp['M000-M001']['XX']['S'][:]

array([8.501775  , 0.19459732, 0.11719746], dtype=float32)

In [ ]:
# Open the Zarr store
root = zarr.open('/home/mukundi/kundikat/visco/visco/output_data/compresseddata.zarr', mode='r')

# Access the 'decompositions' group
decomp_group = root['decompositions']

# List the available baseline groups
baseline_groups = list(decomp_group.keys())
print(f"Available baseline groups: {baseline_groups}")

# Access data for a specific baseline
baseline_key = 'M000-M001'
if baseline_key in decomp_group:
    baseline_group = decomp_group[baseline_key]
    
    # List the available correlations
    correlation_groups = list(baseline_group.keys())
    print(f"Available correlations for baseline {baseline_key}: {correlation_groups}")
    
    # Access data for a specific correlation
    corr_key = 'XX'
    if corr_key in baseline_group:
        corr_group = baseline_group[corr_key]
        
        # Access the decomposition components
        U = corr_group['U'][:]
        singvals = corr_group['singvals'][:]
        WT = corr_group['WT'][:]
        
        # Access other metadata
        baseline_filter = np.array(corr_group.attrs['baseline_filter'])
        ci = corr_group.attrs['ci']
        fullrank = corr_group.attrs['fullrank']
        
        # Do something with the data
        print(f"U shape: {U.shape}")
        print(f"singvals shape: {singvals.shape}")
        print(f"WT shape: {WT.shape}")
        print(f"baseline_filter: {baseline_filter}")
        print(f"ci: {ci}")
        print(f"fullrank: {fullrank}")

In [11]:
maintable = xds_from_table(msfile)[0]#,taql_where=(f"DATA_DESC_ID=0 AND SCAN_NUMBER=1 AND FIELD_ID=0 AND ANTENNA1=0 AND ANTENNA2=1"))[0]
# tabl = table(msfile,readonly=False,lockoptions='auto')
subtable = xds_from_table(mssub)[0]
ant1_mask = (maintable.ANTENNA1.data.compute() == 0)
ant2_mask = (maintable.ANTENNA2.data.compute() == 5)
fieldidmask = (maintable.FIELD_ID.data.compute()==0)
filter = ant1_mask & ant2_mask
baseline_data = maintable.COMPRESSED_DATA.data[:,:,3]
data = maintable.DATA.data[:,:,3]
maintable

<xarray.Dataset> Size: 29MB
Dimensions:          (row: 6048, SV1_DATA-1: 24, SV1_DATA-2: 4, corr: 4,
                      chan: 24, uvw: 3, COMPRESSED_DATA-1: 24,
                      COMPRESSED_DATA-2: 4)
Coordinates:
    ROWID            (row) int64 48kB dask.array<chunksize=(6048,), meta=np.ndarray>
Dimensions without coordinates: row, SV1_DATA-1, SV1_DATA-2, corr, chan, uvw,
                                COMPRESSED_DATA-1, COMPRESSED_DATA-2
Data variables: (12/25)
    TIME_CENTROID    (row) float64 48kB dask.array<chunksize=(6048,), meta=np.ndarray>
    ANTENNA2         (row) int32 24kB dask.array<chunksize=(6048,), meta=np.ndarray>
    SV1_DATA         (row, SV1_DATA-1, SV1_DATA-2) complex64 5MB dask.array<chunksize=(6048, 24, 4), meta=np.ndarray>
    WEIGHT           (row, corr) float32 97kB dask.array<chunksize=(6048, 4), meta=np.ndarray>
    MODEL_DATA       (row, chan, corr) complex64 5MB dask.array<chunksize=(6048, 24, 4), meta=np.ndarray>
    FEED1            (row) int32 24kB dask.array<chunksize=(6048,), meta=np.ndarray>
    ...               ...
    SCAN_NUMBER      (row) int32 24kB dask.array<chunksize=(6048,), meta=np.ndarray>
    SIGMA            (row, corr) float32 97kB dask.array<chunksize=(6048, 4), meta=np.ndarray>
    FLAG_ROW         (row) bool 6kB dask.array<chunksize=(6048,), meta=np.ndarray>
    FLAG             (row, chan, corr) bool 581kB dask.array<chunksize=(6048, 24, 4), meta=np.ndarray>
    ARRAY_ID         (row) int32 24kB dask.array<chunksize=(6048,), meta=np.ndarray>
    OBSERVATION_ID   (row) int32 24kB dask.array<chunksize=(6048,), meta=np.ndarray>
Attributes:
    __daskms_partition_schema__:  ()

In [ ]:
ant = maintable.ANTENNA1.values
ant
zarr_file.create_dataset(name = 'ANTENNA2',data = ant, chunks=(100,))


In [ ]:
baseline_data.compute()[0,20], data.compute()[0,20]

In [ ]:
ranks = [4,5,6,7]

for i in ranks:
    run = svdaverager.uniform_svd(msfile,rank=i,column='DATA',correlation='XX,YY', outcol=f"{i}SV_DATA")